Lukas notebook

In [2]:
!pwd

/Users/lukasfiller/dev/browser-use


In [ ]:
%cd dev
%cd browser-use

/Users/lukasfiller/dev
/Users/lukasfiller/dev/browser-use


In [ ]:
#optional

!git clone https://github.com/browser-use/browser-use
%cd browser-use

/Users/lukasfiller/dev/browser-use


In [ ]:
#creates env
!uv venv --python 3.11

zsh:1: command not found: uv


In [ ]:
#reactivates the env
!source .venv/bin/activate

In [ ]:
#run in cli activated .venv as
#pip install -U browser-use

#or here...run this in the conda env
!uv pip install -U browser-use

In [ ]:
!pip install -U browser-use
!pip install -U browser-use[memory]
!pip install -U langchain-ollama

In [ ]:
!pip install -U playwright

In [5]:
!playwright install

In [7]:

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio

nest_asyncio.apply()

In [ ]:
#needed to run locally
!pip install python-dotenv

from dotenv import load_dotenv
load_dotenv()

  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)


True

END STANDARD STARTUP

In [ ]:
#ollama building block only
#works but still needs to be called

from langchain_ollama import ChatOllama
from browser_use import Agent
from pydantic import SecretStr


# Initialize the model
llm=ChatOllama(model="qwen3:14b", num_ctx=32000)

# Create agent with the model
# Create agent with the model
agent = Agent(
    task='''research how the digital china program is seeking to advance china
    AI capabilities. Use only info found at https://digitalchinawinsthefuture.com/''',
    llm=llm,
    page_extraction_llm=llm
)

INFO     [agent] 🧠 Starting an agent with main_model=qwen3:14b +vision, planner_model=None, extraction_model=None 


/Users/lukasfiller/miniforge3/envs/browser-use/lib/python3.11/site-packages/ollama/_types.py:81: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  if key in self.model_fields:
/Users/lukasfiller/miniforge3/envs/browser-use/lib/python3.11/site-packages/ollama/_types.py:82: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields[key].default is not None


In [ ]:
#incorporate agent call

from langchain_ollama import ChatOllama
from browser_use import Agent
from pydantic import SecretStr

import asyncio

# Initialize the model
llm=ChatOllama(model="qwen3:14b", num_ctx=32000)

async def main():
    agent = Agent(
        task="Compare the price of gpt-4o and DeepSeek-V3",
        llm=llm,
        page_extraction_llm=llm
    )
    result = await agent.run()
    print(result)

asyncio.run(main())

INFO     [agent] 🧠 Starting an agent with main_model=qwen3:14b +vision, planner_model=None, extraction_model=None 


/Users/lukasfiller/miniforge3/envs/browser-use/lib/python3.11/site-packages/ollama/_types.py:81: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  if key in self.model_fields:
/Users/lukasfiller/miniforge3/envs/browser-use/lib/python3.11/site-packages/ollama/_types.py:82: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields[key].default is not None


INFO     [agent] 🚀 Starting task: Compare the price of gpt-4o and DeepSeek-V3
INFO     [agent] 📍 Step 1
AppKit is not available. Make sure you are running this on macOS with pyobjc installed.


/Users/lukasfiller/miniforge3/envs/browser-use/lib/python3.11/site-packages/ollama/_types.py:81: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  if key in self.model_fields:
/Users/lukasfiller/miniforge3/envs/browser-use/lib/python3.11/site-packages/ollama/_types.py:82: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields[key].default is not None


INFO     [agent] 🤷 Eval: Unknown - This is the first step, no previous actions have been taken yet.
INFO     [agent] 🧠 Memory: Starting the task to compare the price of gpt-4o and DeepSeek-V3. Current step: 1/100.
INFO     [agent] 🎯 Next goal: Need to navigate to a website or source that provides pricing information for gpt-4o and DeepSeek-V3.
INFO     [agent] 🛠️  Action 1/1: {"go_to_url":{"url":"https://www.google.com"}}
INFO     [controller] 🔗  Navigated to https://www.google.com
INFO     [agent] 📍 Step 2


...

INFO     [agent] 👍 Eval: Success - The search results page for 'price comparison gpt-4o DeepSeek-V3' was successfully reached. The current page contains relevant information on the pricing of gpt-4o and DeepSeek-V3, including featured snippets and search results from various sources. The interactive elements include links to articles and blog posts that compare the pricing of these models, such as 'DeepSeek V3 vs. OpenAI's GPT-4o: Which AI Model is Better?' and 'DeepSeek V3 vs GPT-4o: Which is Better? - Analytics Vidhya'. These elements can be used to extract detailed pricing information for both models.
INFO     [agent] 🧠 Memory: Step 6/100. Successfully reached the Google search results page for 'price comparison gpt-4o DeepSeek-V3'. The page contains featured snippets and search results that provide pricing information for gpt-4o and DeepSeek-V3. Need to extract the pricing details from these sources.
INFO     [agent] 🎯 Next goal: Extract the pricing information for gpt-4o and DeepS

/Users/lukasfiller/miniforge3/envs/browser-use/lib/python3.11/site-packages/ollama/_types.py:81: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  if key in self.model_fields:
/Users/lukasfiller/miniforge3/envs/browser-use/lib/python3.11/site-packages/ollama/_types.py:82: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields[key].default is not None


INFO     [agent] 👍 Eval: Success - The search results page for 'price comparison gpt-4o DeepSeek-V3' was successfully reached. The current page contains relevant information on the pricing of gpt-4o and DeepSeek-V3, including featured snippets and search results from various sources. The interactive elements include links to articles and blog posts that compare the pricing of these models, such as 'DeepSeek V3 vs. OpenAI's GPT-4o: Which AI Model is Better?' and 'DeepSeek V3 vs GPT-4o: Which is Better? - Analytics Vidhya'. These elements can be used to extract detailed pricing information for both models.
INFO     [agent] 🧠 Memory: Step 7/100. Successfully reached the Google search results page for 'price comparison gpt-4o DeepSeek-V3'. The page contains featured snippets and search results that provide pricing information for gpt-4o and DeepSeek-V3. Need to extract the pricing details from these sources.
INFO     [agent] 🎯 Next goal: Extract the pricing information for gpt-4o and DeepS

/Users/lukasfiller/miniforge3/envs/browser-use/lib/python3.11/site-packages/ollama/_types.py:81: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  if key in self.model_fields:
/Users/lukasfiller/miniforge3/envs/browser-use/lib/python3.11/site-packages/ollama/_types.py:82: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields[key].default is not None


INFO     [agent] 👍 Eval: Success - The featured snippet provides pricing details for both GPT-4o and DeepSeek-V3. GPT-4o charges $1.25 per million input cache hit tokens, $2.50 per million input cache miss tokens, and $10 per million output tokens. DeepSeek-V3 charges $0.07 per million input cache hit tokens, $0.27 per million input cache miss tokens, and $1.10 per million output tokens. This information directly addresses the task of comparing their prices.
INFO     [agent] 🧠 Memory: Step 8/100. Extracted pricing information from the featured snippet on Google's search results page. GPT-4o's pricing is $1.25 per million input cache hit tokens, $2.50 per million input cache miss tokens, and $10 per million output tokens. DeepSeek-V3's pricing is $0.07 per million input cache hit tokens, $0.27 per million input cache miss tokens, and $1.10 per million output tokens. The task is now complete as the pricing comparison has been successfully obtained.
INFO     [agent] 🎯 Next goal: The task 

In [ ]:
#ollama building block only--more complez

from langchain_ollama import ChatOllama
from browser_use import Agent
from pydantic import SecretStr


# Initialize the model
llm=ChatOllama(model="qwen3", num_ctx=32000)
planner_llm = ChatOllama(model='qwen3')

# Create agent with the model
# Create agent with the model
agent = Agent(
    task='''research how the digital china program is seeking to advance china
    AI capabilities. Use only info found at https://digitalchinawinsthefuture.com/''',
    llm=llm,
    planner_llm=planner_llm,           # Separate model for planning
    use_vision_for_planner=False,      # Disable vision for planner
    planner_interval=4
)

scraps below

In [15]:
# -*- coding: utf-8 -*-
import os
import sys
import asyncio
from dotenv import load_dotenv
from langchain_ollama import ChatOllama # Using Ollama

# --- Ensure browser_use is installed or correctly on sys.path ---
try:
    from browser_use import Agent, Browser, BrowserConfig # Import Browser and Config
    print("Successfully imported browser_use components.")
except ImportError as e:
    print(f"Error importing browser_use: {e}")
    print("Please ensure 'browser_use' is installed in your Jupyter kernel's environment.")
    print("You might need to run: pip install browser-use  (or similar, depending on the package name)")
    # Or ensure the path is correct if it's a local module
    # cwd = os.getcwd()
    # print(f"Current working directory: {cwd}")
    # print(f"Python path: {sys.path}")
    raise # Stop execution if import fails

# Load environment variables if needed
load_dotenv()
print("Environment variables loaded (if .env file exists).")

# Initialize the model using Ollama
# --- Make sure your Ollama service is running locally ---
llm = None # Initialize llm to None
try:
    llm = ChatOllama(
        model='qwen3,        # Or 'llama3', 'mistral', etc.
        temperature=0.0,
    )
    print(f"Successfully initialized Ollama model: {llm.model}")
    # Optional: Quick test
    # llm.invoke("test connection")
except Exception as e:
    print(f"Error initializing Ollama model: {e}")
    print("Please ensure the Ollama service is running and the model is available.")
    raise # Stop execution if LLM fails

# --- Explicit Browser Configuration ---
# Note: Playwright browsers might need to be installed ('playwright install' in terminal/cell)
browser = None # Initialize browser to None
try:
    print("Configuring browser...")
    config = BrowserConfig(
        headless=True,  # Run without a visible browser window
        # user_agent="...", # Optional
        # viewport={"width": 1920, "height": 1080}, # Optional
    )
    print("Initializing browser...")
    # This step might download browser drivers on first run
    browser = Browser(config=config)
    print("Browser initialized.")
except Exception as e:
    print(f"Error initializing Browser: {e}")
    print("This might be due to Playwright browsers not being installed.")
    print("Try running '!playwright install' in a cell or 'playwright install' in your terminal.")
    raise # Stop execution if browser fails

# Define the task for the agent
current_year = 2025 # Based on user context
future_date = f"{current_year + 1}-05-01" # Use next year for the search
task = f'Go to kayak.com and find the cheapest flight from Zurich (ZRH) to San Francisco (SFO) departing on {future_date}'
print(f"Task defined: {task}")

# Initialize the Agent - *Pass the browser instance*
print("Initializing Agent...")
if llm and browser:
    agent = Agent(task=task, llm=llm, browser=browser) # Pass browser here
    print("Agent initialized.")
else:
    print("Cannot initialize Agent due to previous errors.")
    raise ValueError("LLM or Browser not initialized successfully.")


# Define the main asynchronous function
async def main():
    print("Running agent asynchronously...")
    result = None # Initialize result
    try:
        result = await agent.run()
        print("Agent run completed.")
        # Process result
        if result and hasattr(result, 'action_results'):
            print("Processing actions...")
            final_content = []
            for action in result.action_results():
                print(f"  Action: {action.action_type}, Status: {action.status}")
                if action.extracted_content:
                    content_snippet = action.extracted_content[:200].replace('\n', ' ') + ('...' if len(action.extracted_content) > 200 else '')
                    print(f"    Extracted: {content_snippet}")
                    final_content.append(action.extracted_content)
            print("\n--- Combined Extracted Content ---")
            print("\n\n".join(final_content) if final_content else "No content extracted.")
            print("--------------------------------")

        elif result:
            print(f"Agent Result (raw): {result}")
        else:
            print("Agent did not return a result.")

    except Exception as e:
        print(f"An error occurred during agent execution: {e}")
        # You might want to re-raise the exception for debugging
        import traceback
        traceback.print_exc()
        # raise
    finally:
        # Ensure the browser used by the agent is closed
        # Since we passed the browser instance, we can close it directly
        if browser and hasattr(browser, 'close'):
            print("Attempting to close browser...")
            try:
                await browser.close()
                print("Browser closed.")
            except Exception as close_e:
                print(f"Error closing browser: {close_e}")
        else:
            print("Browser instance not available for cleanup.")

# Run the main async function in Jupyter
# No need for 'if __name__ == "__main__":' when running directly in a cell
print("Starting async execution...")
try:
    # In modern Jupyter/IPython, you might be able to just 'await main()'
    # but asyncio.run() is generally safer for compatibility.
    asyncio.run(main())
    print("Async execution finished successfully.")
except Exception as e:
    print(f"An critical error occurred running main: {e}")
    # Print traceback if error happens here
    import traceback
    traceback.print_exc()
finally:
    print("Execution block finished.")

SyntaxError: unterminated string literal (detected at line 31) (1222905217.py, line 31)

In [6]:
!ollama run qwen3:14b "What is the capital of France?"

⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠇ ⠋ ⠙ ⠙ ⠸ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠋ ⠋ ⠹ ⠸ ⠼ ⠼ ⠦ ⠦ ⠇ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠴ ⠦ ⠦ ⠧ ⠏ ⠏ ⠙ ⠹ ⠸ ⠸ ⠴ ⠦ ⠦ ⠧ ⠏ ⠏ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠼ ⠴ ⠴ ⠧ ⠇ ⠇ ⠋ ⠋ ⠙ ⠸ ⠸ ⠴ ⠦ ⠧ ⠇ ⠏ ⠏ ⠙ ⠹ ⠸ ⠼ ⠴ ⠦ ⠧ ⠇ ⠏ ⠋ ⠙ ⠹ ⠸ ⠸ ⠴ ⠴ ⠧ ⠇ ⠏ ⠋ ⠙ ⠙ ⠸ ⠸ ⠴ ⠴ ⠦ ⠇ ⠇ ⠏ ⠙ ⠹ ⠸ ⠸ <think>
Okay, so the user is asking for the capital of France. Let me think. I know
know that France is a country in Europe, and its capital is a major city. I
I remember from school that Paris is the capital. But wait, I should double
double-check to make sure I'm not confusing it with another country. For ex
example, Spain's capital is Madrid, and Germany's is Berlin. No, France's i
is definitely Paris. Let me confirm by recalling some landmarks. The Eiffel
Eiffel Tower and the Louvre Museum are in Paris. Yeah, that's right. So the
the answer should be Paris. I don't think there's any other city that's the
the capital of France. Maybe someone might think of Lyon or Marseille, but 


In [ ]:
agent = Agent(
    task="your task",
    llm=llm,
    controller=custom_controller,  # For custom tool calling
    use_vision=True,              # Enable vision capabilities
    save_conversation_path="logs/conversation.json"  # Save chat logs
)

In [ ]:
!pip install -U langchain_groq

In [ ]:
#use groq
import os
from dotenv import load_dotenv
import asyncio
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

from browser_use import Agent
from browser_use.browser.browser import Browser
from browser_use.browser.browser import BrowserConfig
from browser_use.browser.context import BrowserContextConfig, BrowserContextWindowSize

load_dotenv()

os.environ["GROQ_API_KEY"] = "XXXXX"
# Initialize Groq client
#llm=ChatGroq(model='llama-3.3-70b-versatile', api_key=api_key)

llm=ChatGroq(model='llama3-70b-8192', api_key=api_key)

async def run_agent():
    config = BrowserConfig(headless=True,
                           disable_security=True)
    browser = Browser(config)

    agent = Agent(
        task="Who is Lukas Filler?",
        llm=llm,
        browser=browser,
        generate_gif=False,
        use_vision=False
)
    result = await agent.run()
    print("Agent result:", result)

# Run agent asynchronously
asyncio.run(run_agent())


In [ ]:
print("Agent result:", result)


NameError: name 'result' is not defined